In [1]:
#pip install transformers
#pip install --upgrade pip
#pip install tensorflow

In [1]:
#pip install transformers datasets


In [1]:
import getpass
import pandas as pd

In [2]:
#from transformers import pipeline
import psycopg2


In [133]:
# Connect to your local Postgres database
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="postgres",
    user=getpass.getuser(),
    password=getpass.getpass()
)

In [134]:
sql="""

select * from test_sales_data_december

"""

cursor=conn.cursor()
cursor.execute(sql)

In [135]:
df=pd.DataFrame(cursor.fetchall(),columns=['order_id','product','qty','price','date','addr'])
df.head()

,order_id,product,qty,price,date,addr
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


In [136]:
print(df['order_id'].nunique(), df['order_id'].count())

24007 25037


In [137]:
df.shape

(25117, 6)

In [138]:
df.isna().value_counts()

order_id  product  qty    price  date   addr 
False     False    False  False  False  False    25037
True      True     True   True   True   True        80
Name: count, dtype: int64

In [139]:
df['order_id'].notna().value_counts()

order_id
True     25037
False       80
Name: count, dtype: int64

In [140]:
def sql_parser(l):
    operation,count_object,where_clause,condition_var,condition,condition_val,table_name = '','*','','','','',''
    for i in range(0,len(l)):
        if (l[i]=='number'):
            operation="count"
            count_object=l[i+2]
        elif (l[i]=='sum'):
            operation="sum"
            count_object=l[i+2]
        if (l[i]=='with' or l[i]=='where'):
            where_clause=" where "
            condition_var=l[i+1]
            if(l[i+2]=="more"):
                condition=">"
            elif(l[i+2]=="less"):
                condition="<"
            else:
                condition= "="
            condition_val=l[i+4]
        if(l[i]=="from"):
            table_name=l[i+1]
    return (" select "+operation+"("+count_object+")"+" from "+table_name+where_clause+condition_var+" "+condition+" "+condition_val+";")


In [141]:
def text_2_sql(sql_text):
    x = sql_text.split()
    k = sql_parser(x)
    return k
text_2_sql("t number of orders with price more than 1800 from test_sales_data_december")

' select count(orders) from test_sales_data_december where price > 1800;'

In [142]:
def sql_runner(sql):
    cursor=conn.cursor()
    cursor.execute(sql)
    return(cursor.fetchall())

In [143]:

texts = text_2_sql("""number of product from test_sales_data_december""")
print(texts)

 select count(product) from test_sales_data_december  ;


In [144]:
print(sql_runner(texts))


[(25037,)]
